In [33]:
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import image
import matplotlib.pyplot as plt
import tensorflow as tf
from os import listdir


<h3> Load Data </h3>

In [47]:
#Load all images from directory
pokemon_data = []
image_dir = 'images/images'
for filename in listdir(image_dir):
    pokemon_sprite = image.imread(image_dir + '/' + filename)
    pokemon_data.append(pokemon_sprite)
    
# convert list to numpy array  
pokemon_data = np.array(pokemon_data)

#Load labels from csv
label = pd.read_csv('pokemon.csv')
label = label.sort_values(by = 'Name', ascending=True)

# only using primary pokemon type
types = np.array(label['Type1'])



<h3> Normalization and Spliting Data </h3>

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#normalizing data
for img in pokemon_data:
    img = img/255

enc = LabelEncoder()
enc.fit(types)
types = enc.transform(types)


X_train, X_test, y_train, y_test = train_test_split(pokemon_data, types, test_size = 0.3, random_state = 2)



<h3> Data Augmentation </h3>

In [50]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

augmentation_layers = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal_and_vertical"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomTranslation(0.2, 0.2)
])

<h3> Defining Model </h3>

In [55]:
#Custom CNN model
model = tf.keras.Sequential([
    augmentation_layers,
    layers.Conv2D(32, (3, 3), activation = 'relu', input_shape=(120, 120, 4)),
    layers.MaxPooling2D(2, 2),
    
    layers.Conv2D(64, (5, 5), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    
    layers.Conv2D(32, (3, 3), activation = 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.1),
    
    layers.Flatten(),
    layers.Dense(18, activation = 'softmax'),
    
])

In [46]:
y_train.shape

(566,)

<h3> Optimizing and Fitting Model </h3>

In [57]:
from tensorflow.keras.callbacks import EarlyStopping
#model optimization
model.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#early stop when validation accuracy drops
early_stop = EarlyStopping(min_delta = 0.001, patience = 30, restore_best_weights = True)

trained_model = model.fit(X_train, y_train,
                    batch_size = 32,
                    validation_data = (X_test, y_test),
                    shuffle = True,
                    callbacks = [early_stop],
                    epochs = 10
                   )

Epoch 1/10


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_6/conv2d_9/Relu (defined at <ipython-input-57-e9dec539b2a2>:7) ]] [Op:__inference_train_function_3353]

Function call stack:
train_function


<h3> Validation and Prediction </h3>

In [ ]:
plt.plot(trained_model.history['accuracy'], label = 'accuracy')
plt.plot(trained_model.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1])

loss, model_accuracy = model.evaluate(X_val, y_val)

print("Accuracy:", model_accuracy)